# Chapter 4: Tracking code and data versioning

DL models are not just models – **they are intimately tied to the code that trains and tests the model and the data that's used for training and testing**. **If we don't track the code and data that's used for the model, it is impossible to reproduce the model or improve it**. Furthermore, there have been recent industry-wide awakenings and paradigm shifts toward a [data-centric AI](https://www.forbes.com/sites/gilpress/2021/06/16/andrew-ng-launches-a-campaign-for-data-centric-ai/?sh=5cbacdc574f5), where **the importance of data is being lifted to a first-class artifact in building ML and, especially, DL models**. Due to this, we will learn **how to track code and data versioning using MLflow**. We will learn about **the different ways we can track code and pipeline versioning and how to use Delta Lake for data versioning**. By the end of this chapter, you will be able to understand and implement tracking techniques for both code and data with MLflow.

## **Tracking notebook and pipeline versioning**

Data scientists usually start by experimenting with Python notebooks offline, where interactive execution is a key benefit. The success and popularity of Jupyter notebooks are undeniable. However, there are limitations when it comes to using version control for Jupyter notebooks since Jupyter notebooks are stored as JSON data with mixed output and code. This is especially difficult if we trying to track code using MLflow as we're only using Jupyter's native format, whose file extension is .ipynb.

You may not be able to see the exact Git hash in the MLflow tracking server for each run using a Jupyter notebook either. There are a lot of interesting debates on whether or when a Jupyter notebook should be used, especially in a production environment ([see a discussion here](https://medium.com/mlops-community/jupyter-notebooks-in-production-4e0d38803251)). **There are multiple reasons why we shouldn't use Jupyter notebooks in a production environment**, especially when we need reproducibility in an end-to-end pipeline fashion, where unit testing, proper code versioning, and dependency management could be difficult with a lot of notebooks.

There are some early innovations around scheduling, parameterizing, and executing Jupyter notebooks in a workflow fashion using the [open source tool papermill by Netflix](https://papermill.readthedocs.io/en/latest/index.html). However, a recent innovation by Databricks and VS Code makes notebooks much easier to be version controlled and integrated with MLflow. Let's look at the notebook characteristics that were introduced by these two tools:

* **Interactive execution**: Both Databricks's notebooks and VS Code's notebooks can run the same way as traditional Jupyter notebooks, in a cell-by-cell execution mode. By doing this, you can immediately see the output of the results.
* **File format**: Both Databricks's notebooks and VS Code's notebooks are stored as plain-old Python code with a .py file extension. This allows all the regular Python code linting (code format and style checking) to be applied to a notebook.
* **Special symbols for rendering code cells and Mark down cells**: Both Databricks and VS Code leverage some special symbols to render Python files as interactive notebooks. In Databricks, the special symbols to delineate code into different executable cells are as follows:

```python
# COMMAND ----------

import mlflow
import torch
from torchinfo import summary
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.optim import AdamW
from torch.nn.functional import cross_entropy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd
import os
import zipfile
import requests
from tqdm.auto import tqdm
import torchmetrics
```
The code below the special COMMAND line will be rendered as an executable cell in the Databricks web UI portal, as follows:!

![alt text](databricks_exec_file.png)

To add a large chunk of text to describe and comment on the code in Databricks (also known as Markdown cells), you can use the # MAGIC symbol at the beginning of the line, as follows:

```py
# MAGIC %md

# MAGIC #### Notebooks for fine-tuning a pretrained language model to do text-based sentiment classification
```
This is then rendered in the Databricks notebook as a Markdown comment cell.

In VS Code, a slightly different set of symbols is used for these two types of cells. For a code cell, the # %% symbols are used at the beginning of the cell block:

```py
# %%
# --- 1. & 2. Data Download and Preparation ---
data_url = "https://pl-flash-data.s3.amazonaws.com/imdb.zip"
data_path = "./data/"
download_and_extract_zip(data_url, data_path)
```

Given the advantages of Databricks and VS Code notebooks, we suggest using either for version tracking. We can use GitHub to track the versioning of either type of notebook since they use a regular Python file format.

### ***TWO WAYS TO USE DATABRICKS NOTEBOOK VERSION CONTROL***

*For a managed Databricks instance, a notebook version can be tracked in two ways: by looking at the revision history on the side panel of the notebook on the Databricks web UI, or by linking to a remote GitHub repository. Detailed descriptions are available in the Databricks [notebook documentation](https://docs.databricks.com/notebooks/notebooks-use.html#version-control.)*

While the Databricks web portal provides excellent support for notebook version control and integration with MLflow experimentation tracking there is one major drawback of writing code in the Databricks notebook web UI. This is because the web UI is not a typical integrated development environment (IDE) compared to VS Code, where code style and formatting tools. This can have a major impact on code quality and maintainability. Thus, it is highly recommended that you use VS Code to author notebook code.

### ***TWO TYPES OF MLFLOW EXPERIMENTS IN DATABRICKS NOTEBOOKS***

*For a **managed Databricks web portal instance**, there are **two types of MLflow experiments** you can perform: **workspace and notebook experiments**. **A workspace experiment is mainly for a shared experiment folder that is not tied to a single notebook**. Remote code execution can write to a workspace experiment folder if needed. On the other hand, **a notebook scope experiment is tied to a specific notebook** and can be found directly on one of the top-right menu items called Experiment in the notebook page on the Databricks web portal. For more details, please look at the [Databricks documentation website](https://docs.databricks.com/applications/mlflow/tracking.html#experiments).*

Using the `fine_tuning.py`,  you will be able to run it interactively in the VS Code editor and log the experiment in the MLflow Docker server. As a reminder, note that to run this notebook in VS Code successfully, you will need to set up your virtual environment.

## **Pipeline tracking**

Having discussed **notebook code tracking (version and filename)**, let's turn to the topic of pipeline tracking. Before we discuss pipeline tracking, however, we will discuss **the definition of a pipeline in the ML/DL life cycle**. Conceptually, **a pipeline is a multi-step data processing and task workflow. However, the implementation of such a data/task workflow can be quite different**. **A pipeline can be defined as a first-class Python API in some ML packages**. The two most well-known pipeline APIs are as follows:
* [**`sklearn.pipeline.Pipeline`**](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): This is widely used for building tightly integrated multi-step pipelines for classical machine learning or data extract, transform, and load (ETL) pipelines using [`pandas DataFrames`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html).
* [**`pyspark.ml.Pipeline`**](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.Pipeline.html): This is a PySpark version for building simple and tightly integrated multi-step pipelines for machine learning or data ETL pipelines using [`Spark DataFrames`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.html)

However, when we're building a DL model pipeline, **we need to use multiple different Python packages at different steps of the pipeline**, so a one-size-fits-all approach using a single pipeline API doesn't usually work. In addition, neither of the aforementioned pipeline APIs have native support for the current popular DL packages, such as **Huggingface** or **PyTorch**, which require additional integration work. Although some open source DL pipeline APIs exist such as [**Neuraxle**](https://github.com/Neuraxio/Neuraxle), which tries to provide a sklearn-like pipeline interface and framework, it is not widely used. Furthermore, using these API-based pipelines means that you'll be locked in when you need to add more steps to the pipeline, which could reduce your flexibility to extend or evolve a DL pipeline when new requirements arise.

In this book, we will take a different approach to define and build a DL pipeline that's based on [MLflow's MLproject](https://www.mlflow.org/docs/latest/projects.html#mlproject-file) structure. This will give you the most flexibility to build a multi-step pipeline that can be tracked using MLflow. At the same time, for each step, you will be allowed to use the most appropriate DL or data processing packages without being locked in. Let's walk through this by breaking the single file-based Python notebook, `fine_tuning.py`, into a multiple-step pipeline. This pipeline can be visualized as a three-step flow diagram, as shown here:

![Alt text](3_step_pipeline.png)

1. Download the data to a local execution environment
2. Fine-tune the model
3. Register the model

These modular steps may seem to be overkill for our current example, but the power of having a distinctive functional step is evident when more complexities are involved, or when changes are needed at each step. Each step can be modified without them affecting the other steps if we define the parameters that need to be passed between them. Each step is a standalone Python file that can be executed independently with a set of input parameters. There will be a main pipeline Python file that can run the whole pipeline or a sub-section of the pipeline's steps. In the MLproject file, which is a standard YAML file without the file extension, we can define four entry points (`main`, `download_data`, `fine_tuning_model`, and `register_model`), their required input parameters, their types and default values, and the command line to execute each entry point. In our example, these entry points will be provided in a Python command-line execution command. However, you can invoke any kind of execution, such as a batch shell script, if needed for any particular steps. For example, the following lines in the `MLproject` file describe the name of the project, the conda environment definition filename, and the main entry point:

The `entry_points` section lists the first entry point, called main. In the parameters section, there is one parameter called `pipeline_steps`, which allows the user to define a comma-separated list of DL pipeline steps to execute. This parameter is of the `str` type and its default value is all, which means that all the pipeline steps will run. Lastly, the command section lists how to execute this step in the command line.\
The rest of the MLproject file defines the other three pipeline step entry points by following the same syntactic convention as the main entry point.

Now, the next step is **to implement the Python function to execute each step of the pipeline**. So, let's look at the core implementation of the main entry point's Python function, which is called `main.py.` The following lines of code (not the entire Python code in `main.py`) illustrate the core component of implementing the entire pipeline with just one step in the pipeline (`download_data`):

```python
@click.command()
@click.option("--steps", default="all", type=str)
def run_pipeline(steps):
    with mlflow.start_run(run_name='pipeline', nested=True) as active_run:
        download_run = mlflow.run(".", "download_data", parameters={})
if __name__ == "__main__":
    run_pipeline()
```

This main function snippet contains a `run_pipeline` function, which will be run when the `main.py` file is executed in the command line. There is a parameter called `steps`, which will be passed to this function when it's provided. In this example, we are using the [click Python package](https://click.palletsprojects.com/en/8.1.x/) to parse command-line arguments. The `run_pipeline` function **starts an MLflow experiment run by calling `mlflow.start_run` and passing two parameters (`run_name` and `nested`)**. We have used `run_name` before – it's the descriptive phrase for this run. However, the `nested` parameter is new, **which means that this is a parent experiment run**. **This parent experiment run contains some child experiment runs that will be hierarchically tracked in MLflow**. **Each parent run can contain one or more child runs**. In the example code, this contains one step of the pipeline run, called `download_data`, which is invoked by calling `mlflow.run`. This is the key MLflow function to invoke an MLproject's entry point programmatically. Once `download_data` has been invoked and the run has finished, the parent run will also finish, thus concluding the pipeline's run.

### ***TWO WAYS TO EXECUTE AN MLPROJECT'S ENTRY POINT***

*There are two ways to execute an MLproject's entry point. First, you can use MLflow's Python API, known as [`mlflow.run`](https://www.mlflow.org/docs/latest/python_api/mlflow.projects.html#mlflow.projects.run). Alternatively, you can use the MLflow's command-line interface tool, called `mlflow run`, which can be called in a command-line shell environment to [execute any entry point directly](https://www.mlflow.org/docs/latest/cli.html#mlflow-run).*

Now, let's learn how to implement each step in the pipeline generically. For each pipeline step, **we put the Python files in a pipeline folder**. In this example, we have three files: `download_data.py`, `fine_tuning_model.py`, and `register_model.py`. Thus, the relevant files for successfully building an MLflow supported pipeline project are as follows:
```bash
MLproject
conda.yaml
main.py
pipeline/download_data.py
pipeline/fine_tuning_model.py
pipeline/register_model.py
```
For the implementation of each pipeline step, we can use the following Python function templates. A placeholder section is reserved for implementing the actual pipeline step logic:

```python
import click
import mlflow
@click.command()
@click.option("input")
def task(input):
    with mlflow.start_run() as mlrun:
        # Implement pipeline step logic here 
        mlflow.log_parameter('parameter', parameter)
        mlflow.set_tag('pipeline_step', __file__)
        mlflow.log_artifacts(artifacts, artifact_path="data")

if __name__ == '__main__':
    task()
```

This template allows us to standardize the way we implement the pipeline step task. The main idea here is that for each pipeline step task, it needs to start with `mlflow.start_run` to launch an MLflow experiment run. Once we've implemented specific execution logic in the function, we need to log some parameters using `mlflow.log_parameter`, or some artifacts in the artifact store using `mlflow.log_artifacts`, that can be passed to and used by the next step of the pipeline. This is called **pipeline chaining**, and it allows multiple steps of a single pipeline or even different pipelines to share data and artifacts. We also want to set a tag to indicate which step is executed using `mlflow.set_tag`.

For example, in the download_data.py step, the core implementation is as follows:
```python
import click
import mlflow
from flash.core.data.utils import download_data
@click.command()
@click.option("--download_url")
@click.option("--local_folder")
@click.option("--pipeline_run_name")
def task(download_url, local_folder, pipeline_run_name):
    with mlflow.start_run(run_name=pipeline_run_name) as mlrun:
        download_data(download_url, local_folder)
        mlflow.log_param("download_url", download_url)
        mlflow.log_param("local_folder", local_folder)
        mlflow.set_tag('pipeline_step', __file__)
        mlflow.log_artifacts(local_folder, artifact_path="data")
if __name__ == '__main__':
    task()
```

In this `download_data.py` implementation, the task is to download the data for model building from a remote URL to a local folder (`download_data(download_url, local_folder)`). Once we've done this, we will log a few parameters, such as `download_url` and `local_folder`. We can also log the newly downloaded data into the MLflow artifact store using `mlflow.log_artifacts`.

> For this example, this may not seem necessary since we only want to execute the next step in a local development environment. However, for a more realistic scenario in a distributed execution environment where each step could be run in different execution environments, this is very desirable since we only need to pass the artifact URL path to the next step of the pipeline to use; we don't need to know how and where the previous step was executed

In this example, when the `mlflow.log_artifacts(local_folder, artifact_path="data")` statement is called, the downloaded data folder is uploaded to the MLflow artifact store. However, we will not use this artifact path for the downstream pipeline step in this chapter. We will explore how we use this kind of artifact store to pass artifacts to the next step in the pipeline later in this book. **Here, we will use the log parameters to pass the downloaded data path to the next step of the pipeline** (`mlflow.log_param("local_folder", local_folder)`). So, let's look at how we can do that by extending main.py so that it includes the next step, which is the `fine_tuning_model` entry point, as follows:
```python
with mlflow.start_run(run_name='pipeline', nested=True) as active_run:
    download_run = mlflow.run(".", "download_data", parameters={})
    download_run = mlflow.tracking.MlflowClient().get_run(download_run.run_id)
    file_path_uri = download_run.data.params['local_folder']
    fine_tuning_run = mlflow.run(".", "fine_tuning_model", parameters={"data_path": file_path_uri})
```

**We use `mlflow.tracking.MlflowClient().get_run` to get the `download_run` MLflow run object and then use `download_run.data.params` to get `file_path_uri` (in this case, it is just a local folder path)**. This is then passed to the next `mlflow.run`, which is `fine_tuning_run`, as a key-value parameter (`parameters={"data_path": file_path_uri`). This way, the `fine_tuning_run` pipeline step **can use this parameter to prefix its data source path**. This is **a very simplified scenario to illustrate how we can pass data from one step to the next**. Using the `mlflow.tracking.MlflowClient()` API, which is provided by [MLflow](https://www.mlflow.org/docs/latest/python_api/mlflow.tracking.html), makes **accessing a run's information (parameters, metrics, and artifacts) straightforward**.

We can also extend the `main.py` file with the third step of the pipeline by adding the `register_model` step. This time, we need the logged model URI to register a trained model, which depends on `run_id` of the `fine_tuning_model` step. So, in the `fine_tuning_model` step, we need to get the run_id property of fine_tuning_model run and then pass it through the input parameter for the `register_model` run, as follows:
```python
fine_tuning_run_id = fine_tuning_run.run_id
register_model_run = mlflow.run(".", "register_model", parameters={"mlflow_run_id": fine_tuning_run_id})
```

Now, the register_model step can use fine_tuning_run_id to locate the logged model. The core implementation of the register_model step is as follows:
```py
with mlflow.start_run() as mlrun:
    logged_model = f'runs:/{mlflow_run_id}/model'
    mlflow.register_model(logged_model, registered_model_name)
```
This will register a fine-tuned model at the URI defined by the `logged_model` variable to an MLflow model registry.

If you have followed these steps, then you should have a working pipeline that can be tracked by MLflow from end to end. Then run the following command:
```bash
python main.py
```

### **Tracking locally, privately built Python libraries**

ow, let's turn our attention to tracking locally, privately built Python libraries. For publicly released Python libraries, we can explicitly specify their released version, which is published in PyPI, in a requirements file or a conda.yaml file. For example, this chapter's conda.yaml file defines the Python version and provides a reference to a requirements file, as follows:
```YAML
name: dl_model 
channels:
  - conda-forge
dependencies:
  - python=3.8.10
  - pip
  - pip:
    - -r requirements.txt
```
The Python version is defined as 3.8.10 and is being enforced. This conda.yaml file also refers to a requirements.txt file, which contains the following versioned Python packages as a requirements.txt file, which is located in the same directory as the conda.yaml file:
```
ipykernel==6.4.1
lightning-flash[all]==0.5.0
mlflow==1.20.2
transformers==4.9.2
boto3==1.19.7
pytorch-lightning==1.4.9
datasets==1.9.0
click==8.0.3
```

MLflow will use the conda.yaml file and the referenced requirements.txt file to create a conda virtual environment dynamically. This ensures that the execution environment is reproducible and that all the DL model pipelines can be run successfully. You may have noticed that such a virtual environment was created for you the first time you ran the previous section's MLflow pipeline project. You can do this again by running the following command:
```Bash
conda env list
```
This is the virtual environment that's created by MLflow dynamically, which follows the dependencies that are specified in conda.yaml and requirements.txt. Subsequently, when you log the fine-tuned model, conda.yaml and requirements.txt will be automatically logged in the MLflow artifact store.

As we can see, the conda.yaml file was automatically expanded to include the content of requirements.txt, as well as other dependencies that conda decides to include.

For privately built Python packages, which means the Python packages that are not published to PyPI for public consumption and references, the recommended way to include such a Python package is by using `git+ssh`. Let's assume that you have a privately built project called `cool-dl-utils`, that the organization you work for is called `cool_org`, and that your project's repository has been set up in GitHub. If you want to include this project's Python package in the requirements file, you need to make sure that you add your public key to your GitHub settings. If you want to learn how to generate a public key and load it into GitHub, take a look at [GitHub's guide](https://docs.github.com/en/authentication/connecting-to-github-with-ssh/adding-a-new-ssh-key-to-your-github-account). In the `requirements.txt` file, you can add the following line, which will reference a specific GitHub hash (`81218891bbf5a447103884a368a75ffe65b17a44`) and the Python `.egg` package that was built from this private repository (you can also reference a `.whl` package if you wish):

```bash
cool-dl-utils @ git+ssh://git@github.com/cool_org/cool-dl-utils.git@81218891bbf5a447103884a368a75ffe65b17a44#egg=cool-dl-utils
```

If you have a numerically released version in your privately built package, you can also directly reference the release number in the requirements.txt file, as follows:
```bash
git+ssh://git@github.com/cool_org/cool-dl-utils.git@2.11.4
```
Here the release number of `cool-dl-utils` is `2.11.4`. This allows MLflow to pull this privately built package into the virtual environment to execute `MLproject`. In this chapter, we don't need to reference any privately built Python packages, but it is worth noting that MLflow can leverage the `git+ssh` approach to do that.

Now, let's learn how to track data versioning.

 ## **Tracking data versioning in Delta Lake**


We'll learn how data is tracked in MLflow. Historically, data management and versioning are usually considered as being different from machine learning and data science. However, the advent of data-centric AI is playing an increasingly important role, particularly in DL. Therefore, it is critical to know what and how data is being used to improve the DL model. [In the first data-centric AI competition, which was organized by Andrew Ng in the summer of 2021, the requirements to become a winner were not about changing and tuning a model, but rather improving the dataset of a fixed model](https://https-deeplearning-ai.github.io/data-centric-comp/). Here is a quote from the competition's web page:

> "The Data-Centric AI Competition inverts the traditional format and asks you to improve a dataset, given a fixed model. We will provide you with a dataset to improve by applying data-centric techniques such as fixing incorrect labels, adding examples that represent edge cases, applying data augmentation, and so on."

This paradigm shift highlights the importance of data in deep learning, especially supervised deep learning, where labeled data is important. **An implied underlying assumption is that different data will produce different model metrics, even if the same model architecture and parameters are used**. This requires us to diligently track the data versioning process so that we know which version of the data is being used to produce the winning model.

There are several emerging frameworks for tracking data versioning in the ML/DL life cycle. One of the early pioneers in this domain is [DVC](http://dvc.org). It uses a set of GitHub-like commands to pull/push data as if they are code. It allows the data to be stored remotely in S3, or Google Drive, among many other popular stores. However, the data that's stored in the remote store becomes hashed and isn't human-readable. This becomes a locked-in problem since the only way to access the data is through the DVC tool and configuration. In addition, it is hard to track how the data and its schema have been changed. While it is possible to integrate MLflow with DVC, its usability and flexibility are not as desirable as we want. Thus, we will not deep dive into this approach in this book. If you are interested in this, we suggest that you utilize the *Versioning data and models in ML projects using DVC and AWS* reference at the end of this chapter to find more details about using DVC.

The recently open sourced and open format-based [Delta Lake](https://delta.io/) is **a practical solution for integrated data management and version control in a DL/ML project, especially since MLflow can directly support such integration**. This is also the foundational data management layer, called [Lakehouse](https://databricks.com/blog/2020/01/30/what-is-a-data-lakehouse.html), which **unifies both data warehouse and streaming data into one data foundation layer**. It supports both schema change tracking and data versioning, which is ideal or a DL/ML data use scenario. Delta tables are based on the open standard file format called [Parquet](https://parquet.apache.org/), which is widely supported for large-scale data storage.

### ***DELTA TABLE IN DATABRICKS***